## "The mother ship" 
This notebook organizes inpts and function calls for creating the monte carlo csv file

##### Import functions neccessary for calling scripts

In [1]:
# import monte-carlo functions
from get_date import get_date
from get_lat_lon_indices import get_lat_lon_indices
from get_vessel_type import get_vessel_type

# import python functions needed for input variables
from datetime import datetime, date, timedelta
import arrow

IndentationError: unexpected indent (get_date.py, line 22)

##### These are the input variables needed to run the functions for the monte-carlo setup

In [ ]:
# spill date
start_date = datetime(2015, 1, 1, 0, 30)
end_date   = datetime(2018, 12, 31, 23, 55)
delta_time = timedelta(hours = 1)

# spill location
ais_data_year = 2018
n_locations = 1  #desired number of latitude and longitude values
geotiff_directory = '/Users/rmueller/Data/MIDOSS/AIS/ShipTrackDensityGeoTIFFs/'
upsample_factor = 1 # GeoTIFF lat/lon will be sampled at 1/upsample_factor [km] resolution
vessel_types      = ["tanker", "atb", "barge", "cargo", "cruise",
                     "ferry", "fishing", "smallpass", "other"]

##### Imports and definitions for testing scripts and validating output

In [ ]:
# these aren't used or needed for function calls
from rasterio.enums import Resampling
import rasterio as rio

#### Get the spill date

In [ ]:
spill_date = get_date( start_date, 
             end_date, 
             geotiff_directory,
             n_locations,
             delta_time
            )
spill_month = spill_date.month
spill_month_name  = arrow.get(spill_date).format('MMMM') # string
f'{spill_month:02.0f}'

####  Get spill lat/lon value(s) as well as index values for extracting vessel time exposure values from GeoTIFFS

In [ ]:
[lats, lons, x_index, y_index, data_val] = get_lat_lon_indices(geotiff_directory, spill_month, n_locations, upsample_factor)    

In [ ]:
lats

In [ ]:
lons

In [ ]:
x_index

In [ ]:
y_index

In [ ]:
data_val

In [ ]:
# load geotiff and access "data_val" with [x_index, y_index] to verify that the output is equivalent

traffic_reader = rio.open(f'{geotiff_directory}all_2018_{spill_month:02.0f}.tif')

# dataset closes automatically using the method below

with traffic_reader as dataset:
    # resample data to target shape
    data = dataset.read(1,
        out_shape=(
            dataset.count,
            int(dataset.height * upsample_factor),
            int(dataset.width * upsample_factor)
        ),
        resampling=Resampling.bilinear
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )

print(f'compare output data_val = {data_val} hours/km^2 with {data[x_index,y_index]} hours/km^2')    

####  Get vessel type 

In [ ]:
vessel_type = get_vessel_type(geotiff_directory, 
                              vessel_types, 
                              ais_data_year, 
                              n_locations, 
                              spill_month,  # from get_date
                              x_index,      # from get_lat_lon_indices
                              y_index       # from get_lat_lon_indices
                              )
vessel_type